In [1]:
#RUN
from pandas import DataFrame
import pandas as pd
import numpy as np

In [2]:
#RUN
try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode
    
from jsonapi_client import Session, Filter
import pycurl

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [3]:
#RUN
# 'studies' bisa diganti dengan target semisal 'sample' ; 'MGYS00001939' merupakan accession
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00003673').resource

In [4]:
#RUN
#memunculkan study_name
std

<ResourceObject: studies: MGYS00003673 (1730417287120) (1730417287120)>

In [5]:
#RUN
#melihat jumlah sample pada study di atas
std.samples_count

646

In [6]:
#RUN/SKIP
sample = std.samples
#melihat informasi pada salah satu sample yang dipilih
x=sample[0]
x.sample_metadata

[{'key': 'geographic location (longitude)', 'unit': None, 'value': '-105.27'},
 {'key': 'geographic location (country and/or sea,region)',
  'unit': None,
  'value': 'GAZ:United States of America;USA:CO:Boulder'},
 {'key': 'environment (biome)', 'unit': None, 'value': 'urban biome'},
 {'key': 'environment (feature)', 'unit': None, 'value': 'research facility'},
 {'key': 'environment (material)', 'unit': None, 'value': 'sterile water'},
 {'key': 'environmental package', 'unit': None, 'value': 'misc environment'},
 {'key': 'altitude', 'unit': None, 'value': '0.0'},
 {'key': 'depth', 'unit': None, 'value': '0.0'},
 {'key': 'elevation', 'unit': None, 'value': '1624.0'},
 {'key': 'geographic location (latitude)', 'unit': None, 'value': '40.015'},
 {'key': 'instrument model', 'unit': None, 'value': 'Illumina HiSeq 2000'},
 {'key': 'ENA checklist', 'unit': None, 'value': 'ERC000011'},
 {'key': 'last update date', 'unit': None, 'value': '2016-08-19'},
 {'key': 'host', 'unit': None, 'value': 'c

In [ ]:
#SKIP
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

biome_label = 'environment (biome)'
elev_label = 'elevation'
df = DataFrame(columns=('Sample Name', 'Longitude', 'Latitude', 'Elevation', 'Biome', 'Country'))
df.index.name = 'ID Sample'

with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))
        df.loc[sample.accession] = [
            sample.sample_name,
            sample.longitude,
            sample.latitude,
            get_metadata(sample.sample_metadata, elev_label),
            get_metadata(sample.sample_metadata, biome_label),
            sample.geo_loc_name
        ]
df

In [7]:
#RUN
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

#biome_label = 'environment (biome)'
#elev_label = 'elevation'
#df = DataFrame(columns=('Sample Name', 'Longitude', 'Latitude', 'Elevation', 'Biome', 'Country'))
#df.index.name = 'ID Sample'
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []

with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003673',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        acc.append(sample.accession)
        sample_name.append(sample.sample_name)
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))
        analyze.append(b.accession)
        meta = sample.sample_metadata
        meta = pd.DataFrame.from_dict(meta)
        meta = meta.set_index('key') 
        meta = meta.drop(columns='unit')
        meta = meta.transpose()
        df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

analyses: MGYA00261636
analyses: MGYA00261637
analyses: MGYA00261638
analyses: MGYA00261639
analyses: MGYA00261640
analyses: MGYA00261641
analyses: MGYA00261642
analyses: MGYA00261643
analyses: MGYA00261644
analyses: MGYA00261645
analyses: MGYA00261646
analyses: MGYA00261647
analyses: MGYA00261648
analyses: MGYA00261649
analyses: MGYA00261650
analyses: MGYA00261651
analyses: MGYA00261652
analyses: MGYA00261653
analyses: MGYA00261654
analyses: MGYA00261655
analyses: MGYA00261656
analyses: MGYA00261657
analyses: MGYA00261658
analyses: MGYA00261659
analyses: MGYA00261660
analyses: MGYA00261661
analyses: MGYA00261662
analyses: MGYA00261663
analyses: MGYA00261664
analyses: MGYA00261665
analyses: MGYA00261666
analyses: MGYA00261667
analyses: MGYA00261668
analyses: MGYA00261669
analyses: MGYA00261670
analyses: MGYA00261671
analyses: MGYA00261672
analyses: MGYA00261673
analyses: MGYA00261674
analyses: MGYA00261675
analyses: MGYA00261676
analyses: MGYA00261677
analyses: MGYA00261678
analyses: M

In [8]:
df

key,geographic location (longitude),"geographic location (country and/or sea,region)",environment (biome),environment (feature),environment (material),environmental package,altitude,depth,elevation,geographic location (latitude),instrument model,ENA checklist,last update date,host,host scientific name
value,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,0.0,0.0,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
value,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,0.0,0.0,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
value,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,0.0,0.0,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
value,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,0.0,0.0,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
value,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,0.0,0.0,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
value,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,0.0,0.0,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
value,138.937,GAZ:Japan;Japan,cropland biome,alluvial paddy field soil,soil,soil,0.0,0.0,60.0,35.117,Illumina HiSeq 2000,ERC000011,2016-08-19,bulk soil,soil metagenome
value,138.937,GAZ:Japan;Japan,cropland biome,alluvial paddy field soil,soil,soil,0.0,0.0,60.0,35.117,Illumina HiSeq 2000,ERC000011,2016-08-19,bulk soil,soil metagenome
value,138.937,GAZ:Japan;Japan,cropland biome,alluvial paddy field soil,soil,soil,0.0,0.0,60.0,35.117,Illumina HiSeq 2000,ERC000011,2016-08-19,bulk soil,soil metagenome
value,138.937,GAZ:Japan;Japan,cropland biome,alluvial paddy field soil,soil,soil,0.0,0.0,60.0,35.117,Illumina HiSeq 2000,ERC000011,2016-08-19,bulk soil,soil metagenome


In [ ]:
#SKIP
with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))

In [ ]:
#RUN/SKIP
#memunculkan list sample sesuai jumlah
std.samples

In [ ]:
#SKIP
blank = []
for x in sample:
    #print(x.sample_metadata)
    df = pd.DataFrame(x.sample_metadata)
    df['key'].tolist()
    print(df)
    #df = pd.DataFrame(x.sample_metadata)
    #print(df)
    #index = np.where(df['key'].str.contains('location'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('biome'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('elevation'))[0]
    #blank.append(df.iloc[index, :])
    #df_new = pd.concat(blank)
#df_new

In [ ]:
#RUN
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00000837').resource
print(std.accession)
#melihat informasi pada salah satu sample yang dipilih
#sample = std.samples

In [ ]:
#SKIP
for x in sample:
    print('sample: '+str(x.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#SKIP
#membuat container kosong di bawah :
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []
for x in sample:
    acc.append(x.accession)
    sample_name.append(x.sample_name)
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
    analyze.append(b.accession)
    meta = x.sample_metadata
    meta = pd.DataFrame.from_dict(meta)
    meta = meta.set_index('key') 
    meta = meta.drop(columns='unit')
    meta = meta.transpose()
    df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

In [9]:
#RUN
#reset index agar dapat di merge
df = df.reset_index()
df = df.drop(columns='index')
df = df.drop(columns='altitude')
df = df.drop(columns='depth')
#df = df.drop(columns='level_0')

In [10]:
#RUN
#script merge
merged = acc.merge(sample_name, left_index=True, right_index=True, how='inner')
merged = merged.merge(analyze, left_index=True, right_index=True, how='inner')
merged = merged.merge(df, left_index=True, right_index=True, how='inner')
#df
merged

,ID_Sample,Sample Name,ID_Analyses,geographic location (longitude),"geographic location (country and/or sea,region)",environment (biome),environment (feature),environment (material),environmental package,elevation,geographic location (latitude),instrument model,ENA checklist,last update date,host,host scientific name
0,ERS1300624,control sample 1BLANK,MGYA00261636,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
1,ERS1300625,control sample 2BLANK,MGYA00261637,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
2,ERS1300626,control sample 3BLANK,MGYA00261638,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
3,ERS1300627,control sample 4BLANK,MGYA00261639,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
4,ERS1300628,control sample 5BLANK,MGYA00261640,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
5,ERS1300629,control sample 6BLANK,MGYA00261641,-105.27,GAZ:United States of America;USA:CO:Boulder,urban biome,research facility,sterile water,misc environment,1624.0,40.015,Illumina HiSeq 2000,ERC000011,2016-08-19,control,metagenome
6,ERS1300630,Bulk soil unknown MS00001,MGYA00261642,138.937,GAZ:Japan;Japan,cropland biome,alluvial paddy field soil,soil,soil,60.0,35.117,Illumina HiSeq 2000,ERC000011,2016-08-19,bulk soil,soil metagenome
7,ERS1300631,Bulk soil unknown MS00002,MGYA00261643,138.937,GAZ:Japan;Japan,cropland biome,alluvial paddy field soil,soil,soil,60.0,35.117,Illumina HiSeq 2000,ERC000011,2016-08-19,bulk soil,soil metagenome
8,ERS1300632,Bulk soil unknown MS00003,MGYA00261644,138.937,GAZ:Japan;Japan,cropland biome,alluvial paddy field soil,soil,soil,60.0,35.117,Illumina HiSeq 2000,ERC000011,2016-08-19,bulk soil,soil metagenome
9,ERS1300633,Bulk soil unknown MS00004,MGYA00261645,138.937,GAZ:Japan;Japan,cropland biome,alluvial paddy field soil,soil,soil,60.0,35.117,Illumina HiSeq 2000,ERC000011,2016-08-19,bulk soil,soil metagenome


In [51]:
#RUN SEKALI AJA
#membuat folder
from pathlib import Path
import os
#_dir = os.getcwd()
_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Padi')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)

D:\Seminar\Seminar_Anam\Study_Padi


In [13]:
#RUN
import os
#Set default cwd
#os.chdir('D:\\Seminar\\Seminar_Anam')
#print(os.getcwd())
#masuk ke folder "Study_Jagung/Padi"
%cd Study_Padi
#export ke .csv
merged.to_csv(std.accession+'.csv')

D:\Seminar\Seminar_Anam\Study_Padi


In [ ]:
#SKIP
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#SKIP
#memasukkan sample yang dipilih ke bentuk kolom
df = pd.DataFrame(sample[0].sample_metadata)
df

In [ ]:
#SKIP
blank = []

In [ ]:
#SKIP
#membuat index baru dari list 'key' yang dipilih
index = np.where(df['key'].str.contains('location'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('biome'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('elevation'))[0]
blank.append(df.iloc[index, :])

In [ ]:
#SKIP
#'key' yang dilih dijadikan data frame baru
df_new = pd.concat(blank)
df_new

In [ ]:
#SKIP
filename_sample = '' 

In [ ]:
#SKIP
#membuat loop accession di dalam bentuk list
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
            #for z in y.downloads:
                #if 'json.biom' in z.url:
                    #print(z.url)

In [ ]:
#SKIP
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)
#%rmdir(Study+str)
#with open('_dir'+i.accession+'.biom', 'wb') as f:
    #c = pycurl.Curl()
    #c.setopt(c.URL, z.url)
    #c.setopt(c.WRITEDATA, f)
    #c.perform()
#c.close()